In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [8]:
df_status_declare = pd.read_excel('content/Данные от экспертов.xlsx', sheet_name="Статус по простоям (июль 2024)", header=10)

In [9]:
df_status_declare = df_status_declare[1:]

In [10]:
time_filter = lambda x: x[:-3]
df_status_declare['Время начала интервала'] = df_status_declare['Время начала интервала'].apply(time_filter)
df_status_declare['Время окончания интервала'] = df_status_declare['Время окончания интервала'].apply(time_filter)

In [12]:
df_raw_data = pd.read_excel('content/Данные от экспертов.xlsx', sheet_name="Raw data (июль 2024)", header=9)

In [13]:
df_raw_data = df_raw_data[1:]
df_raw_data

,Unnamed: 0,Время начала интервала,Unnamed: 2,Время окончания интервала,Длительность,Линия,Оборудование,Unnamed: 7,Статус,Unnamed: 9,Заказ,Unnamed: 11,SKU
1,NaN,01/07/2024 08:00,NaN,01/07/2024 08:13,13.4,CL4,Блок розлива (CL4),NaN,Производство > 95%,NaN,NaN,NaN,NaN
2,NaN,01/07/2024 08:13,NaN,01/07/2024 08:13,0.183333,CL4,Блок розлива (CL4),NaN,Вмешательство оператора\n,NaN,NaN,NaN,NaN
3,NaN,01/07/2024 08:13,NaN,01/07/2024 08:18,4.433333,CL4,Блок розлива (CL4),NaN,Производство > 75% < 95%,NaN,NaN,NaN,NaN
4,NaN,01/07/2024 08:18,NaN,01/07/2024 09:17,59.1,CL4,Блок розлива (CL4),NaN,Производство > 95%,NaN,NaN,NaN,NaN
5,NaN,01/07/2024 09:17,NaN,01/07/2024 09:18,1.016666,CL4,Блок розлива (CL4),NaN,Внешняя ошибка,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,NaN,31/07/2024 00:18,NaN,31/07/2024 00:48,30.016666,CL4,Блок розлива (CL4),NaN,Производство > 95%,NaN,NaN,NaN,NaN
6641,NaN,31/07/2024 00:48,NaN,31/07/2024 00:50,2,CL4,Блок розлива (CL4),NaN,Производство > 75% < 95%,NaN,NaN,NaN,NaN
6642,NaN,31/07/2024 00:50,NaN,31/07/2024 01:21,31.516666,CL4,Блок розлива (CL4),NaN,Производство > 95%,NaN,NaN,NaN,NaN
6643,NaN,31/07/2024 01:21,NaN,31/07/2024 01:22,0.316666,CL4,Блок розлива (CL4),NaN,Производство > 75% < 95%,NaN,NaN,NaN,NaN


In [14]:
df_raw_data[df_raw_data['Время начала интервала'] == '01/07/2024 11:35']

,Unnamed: 0,Время начала интервала,Unnamed: 2,Время окончания интервала,Длительность,Линия,Оборудование,Unnamed: 7,Статус,Unnamed: 9,Заказ,Unnamed: 11,SKU
36,NaN,01/07/2024 11:35,NaN,01/07/2024 11:55,20.416666,CL4,Блок розлива (CL4),NaN,Вмешательство оператора\n,NaN,NaN,NaN,NaN


In [15]:
df_raw_data[df_raw_data['Время начала интервала'] == '01/07/2024 11:55']

,Unnamed: 0,Время начала интервала,Unnamed: 2,Время окончания интервала,Длительность,Линия,Оборудование,Unnamed: 7,Статус,Unnamed: 9,Заказ,Unnamed: 11,SKU
37,NaN,01/07/2024 11:55,NaN,01/07/2024 11:59,3.6,CL4,Блок розлива (CL4),NaN,Производство < 75%,NaN,NaN,NaN,NaN


In [16]:
_merged = df_status_declare.merge(df_raw_data, on='Время начала интервала')
_merged

,Линия_x,Unnamed: 1,Дата,Смена,Время начала интервала,Время окончания интервала_x,Длительность_x,Unnamed: 7_x,Плановая длительность,Unnamed: 9_x,...,Время окончания интервала_y,Длительность_y,Линия_y,Оборудование_y,Unnamed: 7_y,Статус,Unnamed: 9_y,Заказ,Unnamed: 11,SKU
0,CL4,NaN,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 11:35,01/07/2024 11:55,20.416666,NaN,NaN,NaN,...,01/07/2024 11:55,20.416666,CL4,Блок розлива (CL4),NaN,Вмешательство оператора\n,NaN,NaN,NaN,NaN
1,CL4,NaN,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 15:52,01/07/2024 16:14,21.7,NaN,NaN,NaN,...,01/07/2024 15:52,0.633333,CL4,Блок розлива (CL4),NaN,Производство < 75%,NaN,NaN,NaN,NaN
2,CL4,NaN,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 15:52,01/07/2024 16:14,21.7,NaN,NaN,NaN,...,01/07/2024 16:14,21.7,CL4,Блок розлива (CL4),NaN,Вмешательство оператора\n,NaN,NaN,NaN,NaN
3,CL4,NaN,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 17:52,01/07/2024 18:21,29.433333,NaN,NaN,NaN,...,01/07/2024 18:21,29.433333,CL4,Блок розлива (CL4),NaN,Затор на выходе,NaN,NaN,NaN,NaN
4,CL4,NaN,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 18:22,01/07/2024 18:31,8.733333,NaN,NaN,NaN,...,01/07/2024 18:31,8.733333,CL4,Блок розлива (CL4),NaN,Затор на выходе,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,CL4,NaN,30/07/2024,Ночная (20:00-08:00 ),30/07/2024 21:05,30/07/2024 21:09,3.916666,NaN,NaN,NaN,...,30/07/2024 21:05,0.183333,CL4,Блок розлива (CL4),NaN,Производство > 75% < 95%,NaN,NaN,NaN,NaN
425,CL4,NaN,30/07/2024,Ночная (20:00-08:00 ),30/07/2024 21:05,30/07/2024 21:09,3.916666,NaN,NaN,NaN,...,30/07/2024 21:09,3.916666,CL4,Блок розлива (CL4),NaN,Внешняя ошибка,NaN,NaN,NaN,NaN
426,CL4,NaN,30/07/2024,Ночная (20:00-08:00 ),30/07/2024 21:42,31/07/2024 00:08,146.95,NaN,NaN,NaN,...,31/07/2024 00:08,146.95,CL4,Блок розлива (CL4),NaN,Нехватка на входе,NaN,NaN,NaN,NaN
427,CL4,NaN,30/07/2024,Ночная (20:00-08:00 ),31/07/2024 00:09,31/07/2024 00:11,2.35,NaN,NaN,NaN,...,31/07/2024 00:11,2.35,CL4,Блок розлива (CL4),NaN,Вмешательство оператора\n,NaN,NaN,NaN,NaN


In [18]:
_merged.to_excel('content/merged.xlsx')

# Текстовый анализ

In [ ]:
df = pd.read_excel('/content/Данные от экспертов.xlsx')

In [ ]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
352,4.383333,NaN,Санитарные регламенты,Внешние мойки,Ополаскивание укупора и розлива
353,3.916666,NaN,Организационные,Проверки по качеству,Лабораторные проверки
354,146.95,NaN,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт
355,2.35,NaN,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/beer_features.xlsx', header=3).dropna(axis=1, how='all')
main_features = pd.read_excel('/content/beer_features_main.xlsx', header=3).dropna(axis=1, how='all')

In [ ]:
main_features['Статус'] =  main_features['Статус'].astype('category').cat.codes

In [ ]:
from datetime import datetime

main_features['Время начала интервала'] = main_features['Время начала интервала'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M'))
main_features['День недели'] = main_features['Время начала интервала'].dt.weekday

In [ ]:
def categorize_time(hour):
    if 5 <= hour < 12:
        return 0
    elif 12 <= hour < 17:
        return 1
    elif 17 <= hour < 22:
        return 2
    else:
        return 3

# Apply the function to create a new column with time categories
main_features['Время дня'] = main_features['Время начала интервала'].apply(lambda x: x.hour).apply(categorize_time)

In [ ]:
df

,Линия,Дата,Смена,Время начала интервала,Время окончания интервала,Длительность,Простои,Причина,Общее описание,код причины,Детальное описание,Оборудование (ID в Монолите),Оборудование,Компонент,№ тех заявки (PMN заявки),Симптом,Код причины,Комментарий PMN,Статус тех заявки (PNM)
0,CL4,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 11:35:08,01/07/2024 11:55:33,20.416666,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,1000132103,Блок розлива,Розлив,50211857,Симптом,1.000132e+09,переход,sent
1,CL4,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 15:52:59,01/07/2024 16:14:41,21.700000,Переналадки,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,2214102,переход,1000132103,Блок розлива,Розлив,50211878,Симптом,1.000132e+09,переход,sent
2,CL4,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 17:52:22,01/07/2024 18:21:48,29.433333,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,1000132103,Блок розлива,Розлив,50211893,Симптом,1.000132e+09,нет сьема,sent
3,CL4,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 18:22:22,01/07/2024 18:31:06,8.733333,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,1000132103,Блок розлива,Розлив,NaN,NaN,NaN,NaN,NaN
4,CL4,01/07/2024,Дневная (08:00-20:00 ),01/07/2024 18:51:42,01/07/2024 18:54:03,2.350000,Прочие внеплановые простои,Загруженность склада,Съем ГП,2112001,нет сьема,1000132103,Блок розлива,Розлив,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,CL4,30/07/2024,Дневная (08:00-20:00 ),30/07/2024 19:11:29,30/07/2024 19:15:52,4.383333,Санитарные регламенты,Внешние мойки,Ополаскивание укупора и розлива,2310502,Обл.гор. водой,1000132103,Блок розлива,Автомат укупорочный,NaN,NaN,NaN,NaN,NaN
342,CL4,30/07/2024,Ночная (20:00-08:00 ),30/07/2024 21:05:44,30/07/2024 21:09:39,3.916666,Организационные,Проверки по качеству,Лабораторные проверки,2512701,Отбор проб.,1000132103,Блок розлива,Розлив,50214882,Симптом,1.000132e+09,Отбор проб.,sent
343,CL4,30/07/2024,Ночная (20:00-08:00 ),30/07/2024 21:42:00,31/07/2024 00:08:57,146.950000,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт,2214201,Переход на FLash. СИП. АПМ.,1000132103,Блок розлива,Розлив,50214887,Симптом,1.000132e+09,Переход на FLash. СИП. АПМ.,sent
344,CL4,30/07/2024,Ночная (20:00-08:00 ),31/07/2024 00:09:34,31/07/2024 00:11:55,2.350000,Переналадки,Переход через щ. СИП,Продукт СИП щ. продукт,2214201,Переход на FLash. СИП. АПМ.,1000132103,Блок розлива,Розлив,NaN,NaN,NaN,NaN,NaN


In [ ]:
main_features

,Время начала интервала,Время окончания интервала,Длительность,Линия,Оборудование,Статус,День недели,Время дня
0,2024-07-01 08:00:00,01/07/2024 08:13,13.400000,CL4,Блок розлива (CL4),10,0,0
1,2024-07-01 08:13:00,01/07/2024 08:13,0.183333,CL4,Блок розлива (CL4),1,0,0
2,2024-07-01 08:13:00,01/07/2024 08:18,4.433333,CL4,Блок розлива (CL4),9,0,0
3,2024-07-01 08:18:00,01/07/2024 09:17,59.100000,CL4,Блок розлива (CL4),10,0,0
4,2024-07-01 09:17:00,01/07/2024 09:18,1.016666,CL4,Блок розлива (CL4),2,0,0
...,...,...,...,...,...,...,...,...
6639,2024-07-31 00:18:00,31/07/2024 00:48,30.016666,CL4,Блок розлива (CL4),10,2,3
6640,2024-07-31 00:48:00,31/07/2024 00:50,2.000000,CL4,Блок розлива (CL4),9,2,3
6641,2024-07-31 00:50:00,31/07/2024 01:21,31.516666,CL4,Блок розлива (CL4),10,2,3
6642,2024-07-31 01:21:00,31/07/2024 01:22,0.316666,CL4,Блок розлива (CL4),9,2,3


In [ ]:
sum(df['Длительность']) / sum(main_features['Длительность'])

0.28704206373781055

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(main_features[['Длительность', 'День недели', 'Время дня']], main_features['Статус'], test_size=0.2, random_state=42)

In [ ]:
# !pip install catboost
from catboost import CatBoostClassifier
model = CatBoostClassifier(verbose=False) # Suppress verbose output

# Fit model
model.fit(X_train, y_train, cat_features=['День недели', 'Время дня'])

# Get predictions
preds = model.predict(X_test).flatten()
print(preds)

[ 9 10  8 ... 10  8  9]


In [ ]:
from sklearn.metrics import f1_score
#не кодируя 'День недели', 'Время дня' как категориальные признаки
print(f1_score(y_test, preds, average='macro'))
print(f1_score(y_test, preds, average='micro'))

0.20296031988219954
0.5003762227238525


In [ ]:
#кодируя 'День недели', 'Время дня' как категориальные признаки
print(f1_score(y_test, preds, average='macro'))
print(f1_score(y_test, preds, average='micro'))

0.18501793272690212
0.5116629044394282


In [ ]:
prostoi = pd.get_dummies(df['Простои'])
prichina = pd.get_dummies(df['Причина'])
obshee_opisanie = pd.get_dummies(df['Общее описание'])
component = pd.get_dummies(df['Компонент'])
# print(len(df['Простои'].unique()))
# print(len(df['Причина'].unique()))
# print(len(df['Общее описание'].unique()))
# print(len(df['Детальное описание'].unique()))
# print(len(df['Компонент'].unique()))


146
3


In [ ]:
obshee_opisanie

,АПМ (Пенная санитарная обработка),Дезинфекция,Изм-е дизайна (цвета/печати/акциз),Лабораторные проверки,Малые остановки,Ополаскивание укупора и розлива,Отсутствие заявок,ПТО,Переход продукт вода продукт,Переход продукт вода продукт + АПМ,...,Прогон в 00:00 часов,Продукт СИП щ-к продукт,Продукт СИП щ. продукт,Продукт стерил. продукт,Смена дизайна (экспорт/промо),Смена расходных материалов,Съем ГП,"Та же машина, разное кол-во рядов, разное кол-во шт.",Тесты материалов,Услуги подрядных организаций
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
342,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
343,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
344,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
